# Create a simple transmission line

## Perform imports and define constants

Perform required imports.

In [1]:
import json
import os
import tempfile

from pyedb import Edb

Define constants.

In [2]:
AEDT_VERSION = "2025.2"
NG_MODE = False

## Create an empty design

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")
edb_path = os.path.join(temp_folder.name, "simple_transmission_line.aedb")
edbapp = Edb(edb_path, edbversion=AEDT_VERSION)

C:\actions-runner\_work\pyaedt-examples\pyaedt-examples\.venv\lib\site-packages\pyedb\misc\decorators.py:55: UserWarning: Argument `edbversion` is deprecated for method `Edb`; use `version` instead.
  warnings.warn(
C:\actions-runner\_work\pyaedt-examples\pyaedt-examples\.venv\lib\site-packages\pyedb\generic\design_types.py:301: UserWarning: Your ANSYS AEDT version is eligible to gRPC version.You might consider switching to that version for better user experience.For more information please check this link: https://edb.docs.pyansys.com/version/dev/grpc_api/index.html
  warnings.warn(GRPC_GENERAL_WARNING, UserWarning)


PyEDB INFO: Star initializing Edb 10:31:01.161149


PyEDB INFO: Edb version 2025.2


PyEDB INFO: Logger is initialized. Log file is saved to C:\Users\ansys\AppData\Local\Temp\pyedb_ansys.log.


PyEDB INFO: legacy v0.67.1


PyEDB INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


PyEDB INFO: create_edb completed in 8.5544 seconds.


PyEDB INFO: EDB C:\Users\ansys\AppData\Local\Temp\tmpr25sah3w.ansys\simple_transmission_line.aedb created correctly.


PyEDB INFO: EDB initialization completed in 8.6182 seconds.


In [4]:
# Create a default 2-layer stackup
edbapp.stackup.create_symmetric_stackup(layer_count=2, soldermask=False)

True

## Create config file

In [5]:
cfg = {"stackup": {}, "modeler": {}, "variables": {}}

cfg["variables"] = [{"name": "trace_width", "value": "0.2mm"}]

cfg["stackup"]["materials"] = [
    {"name": "copper", "permittivity": 1, "conductivity": 58000000.0},
    {"name": "fr4", "permittivity": 3.77, "dielectric_loss_tangent": 0.005},
    {"name": "solder_resist", "permittivity": 3.0, "dielectric_loss_tangent": 0.035},
]

cfg["stackup"]["layers"] = [
    {"name": "TOP", "type": "signal", "material": "copper", "fill_material": "solder_resist", "thickness": "0.035mm"},
    {"name": "DE", "type": "dielectric", "material": "fr4", "fill_material": "", "thickness": "0.5mm"},
    {
        "name": "BOTTOM",
        "type": "signal",
        "material": "copper",
        "fill_material": "solder_resist",
        "thickness": "0.035mm",
    },
]

cfg["modeler"]["traces"] = [
    {
        "name": "trace_1",
        "layer": "TOP",
        "width": "trace_width",
        "path": [[0, 0], [0, "3mm"]],
        "net_name": "SIG",
        "start_cap_style": "flat",
        "end_cap_style": "flat",
    },
    {
        "name": "trace_1_void",
        "layer": "TOP",
        "width": "trace_width+0.3mm",
        "path": [[0, 0], [0, "3mm"]],
        "net_name": "GND",
    },
]
cfg["modeler"]["planes"] = [
    {
        "type": "rectangle",
        "name": "GND_TOP",
        "layer": "TOP",
        "net_name": "GND",
        "lower_left_point": ["-2mm", 0],
        "upper_right_point": ["2mm", "3mm"],
        "voids": ["trace_1_void"],
    },
    {
        "type": "rectangle",
        "name": "GND_BOTTOM",
        "layer": "BOTTOM",
        "net_name": "GND",
        "lower_left_point": ["-2mm", 0],
        "upper_right_point": ["2mm", "3mm"],
    },
]

cfg["modeler"]["padstack_definitions"] = [
    {
        "name": "via",
        "hole_plating_thickness": "0.025mm",
        "material": "copper",
        "pad_parameters": {
            "regular_pad": [
                {
                    "layer_name": "TOP",
                    "shape": "circle",
                    "diameter": "0.5mm",
                },
                {
                    "layer_name": "BOTTOM",
                    "shape": "circle",
                    "diameter": "0.5mm",
                },
            ],
        },
        "hole_range": "through",
        "hole_parameters": {
            "shape": "circle",
            "diameter": "0.25mm",
        },
    }
]

cfg["modeler"]["padstack_instances"] = [
    {
        "name": "gvia_l_1",
        "definition": "via",
        "layer_range": ["TOP", "BOTTOM"],
        "position": ["-1mm", "0.5mm"],
        "net_name": "GND",
    },
    {
        "name": "gvia_l_2",
        "definition": "via",
        "layer_range": ["TOP", "BOTTOM"],
        "position": ["-1mm", "1.5mm"],
        "net_name": "GND",
    },
    {
        "name": "gvia_l_3",
        "definition": "via",
        "layer_range": ["TOP", "BOTTOM"],
        "position": ["-1mm", "2.5mm"],
        "net_name": "GND",
    },
    {
        "name": "gvia_r_1",
        "definition": "via",
        "layer_range": ["TOP", "BOTTOM"],
        "position": ["1mm", "0.5mm"],
        "net_name": "GND",
    },
    {
        "name": "gvia_r_2",
        "definition": "via",
        "layer_range": ["TOP", "BOTTOM"],
        "position": ["1mm", "1.5mm"],
        "net_name": "GND",
    },
    {
        "name": "gvia_r_3",
        "definition": "via",
        "layer_range": ["TOP", "BOTTOM"],
        "position": ["1mm", "2.5mm"],
        "net_name": "GND",
    },
]

cfg["ports"] = [
    {
        "name": "wport_1",
        "type": "wave_port",
        "primitive_name": "trace_1",
        "point_on_edge": [0, 0],
        "horizontal_extent_factor": 10,
        "vertical_extent_factor": 2,
        "pec_launch_width": "0,2mm",
    },
    {
        "name": "wport_2",
        "type": "wave_port",
        "primitive_name": "trace_1",
        "point_on_edge": [0, "3mm"],
        "horizontal_extent_factor": 10,
        "vertical_extent_factor": 2,
        "pec_launch_width": "0,2mm",
    },
]

Create the config file.

In [6]:
file_json = os.path.join(temp_folder.name, "cfg.json")
with open(file_json, "w") as f:
    json.dump(cfg, f, indent=4, ensure_ascii=False)

Apply configuration to the example layout

In [7]:
edbapp.configuration.load(config_file=file_json)
edbapp.configuration.run()

PyEDB INFO: Updating nets finished. Time lapse 0:00:00


PyEDB INFO: Updating components finished. Time lapse 0:00:00


PyEDB INFO: Creating pin groups finished. Time lapse 0:00:00


PyEDB INFO: Placing sources finished. Time lapse 0:00:00


PyEDB INFO: Applying materials finished. Time lapse 0:00:00


...

PyEDB INFO: Placing ports finished. Time lapse 0:00:00.015875


PyEDB INFO: Placing terminals completed in 0.0000 seconds.


PyEDB INFO: Placing probes finished. Time lapse 0:00:00


PyEDB INFO: Applying operations completed in 0.0000 seconds.


True

Save and close EDB.

In [8]:
edbapp.save()
edbapp.close()

PyEDB INFO: Save Edb file completed in 0.0000 seconds.


PyEDB INFO: Close Edb file completed in 0.0000 seconds.


True

## Uncomment to load edb into HFSS 3D Layout.

h3d = Hfss3dLayout(edbapp.edbpath, version=AEDT_VERSION, non_graphical=NG_MODE, new_desktop=True)

## Clean up

All project files are saved in the folder ``temp_folder.name``.
If you've run this example as a Jupyter notebook, you
can retrieve those project files. The following cell
removes all temporary files, including the project folder.

In [9]:
temp_folder.cleanup()